In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.stats as stats
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider, IntText, FloatSlider, FloatText, BoundedFloatText, Checkbox
import ipywidgets as widgets
from IPython.display import display
import statsmodels.api as sm

my_blue = sb.color_palette("tab10")[0]
my_orange = sb.color_palette("tab10")[1]

style = {'description_width': 'initial'}
layout = {'width': '1000px'}
pad = {"margin": '0px 0px 20px 0px', 'width': '1000px'}

from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 10000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

PEwidget1 = interact_manual(base_pay = IntText(value=30000, description='Base Pay', 
                                              style=style, layout=layout),
           epsilon = FloatText(value=4000, step=0.01,
                                      description='Noise in Pay (mean of random error; should be positive or zero)', 
                                      style=style, layout=pad),
                 
           n_men_job_1=IntSlider(value=120, min=0, max=10000, 
                                 description='N Men in Job 1', 
                                 style=style, layout=pad), 
           n_women_job_1=IntSlider(value=80, min=0, max=10000, 
                                   description='N Women in Job 1', style=style, 
                                   layout=layout),

           n_men_job_2 = IntSlider(value=110, min=0, max=10000, 
                                   description='N Men in Job 2', style=style, 
                                   layout=layout), 
           n_women_job_2 = IntSlider(value=90, min=0, max=10000, 
                                     description='N Women in Job 2', 
                                     style=style, layout=layout),
           extra_pay_job_2=FloatText(value=10000, 
                                     description='Pay Bonus for Job 2 (relative to Job 1)', 
                                     style=style, layout=pad),

           n_men_job_3 = IntSlider(value=70, min=0, max=10000, 
                                   description='N Men in Job 3', style=style, 
                                   layout=layout), 
           n_women_job_3 = IntSlider(value=130, min=0, max=10000, 
                                     description='N Women in Job 3', 
                                     style=style, layout=layout),
           extra_pay_job_3=FloatText(value=20000, style=style, layout=pad,
                                     description='Pay Bonus for Job 3 (relative to Job 1)'),

           p_men_A_j1=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 1 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j1=FloatSlider(value=0.5, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 1 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j1=FloatText(value=15000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 1'),

           p_men_A_j2=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 2 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j2=FloatSlider(value=0.5, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 2 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j2=FloatText(value=15000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 2'),

           p_men_A_j3=FloatSlider(value=0.2, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 3 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j3=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 3 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j3=FloatText(value=15000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 3'),

           eval_men_A_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank A'),
           eval_men_A_sd = FloatText(value=1,style=style, layout=layout,description='Noise in Performance for Men in Rank A (standard deviation; should be positive)'),
           eval_women_A_mu=FloatText(value=1, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank A'),
           eval_women_A_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank A (standard deviation; should be positive)'),
           extra_pay_eval_A =FloatText(value=5000, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for A Rank'),

           eval_men_B_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank B'),
           eval_men_B_sd=FloatText(value=1, 
                                          style=style, layout=layout,
                                          description='Noise in Performance for Men in Rank B (standard deviation; should be positive)'),
           eval_women_B_mu=FloatText(value=1, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank B'),
           eval_women_B_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank B (standard deviation; should be positive)'),
           extra_pay_eval_B =FloatText(value=5000, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for B Rank'),

           include_job_controls=Checkbox(value=True,description="Include Job Controls?", 
                                         style=style, layout=layout),
           include_rank_control=Checkbox(value=True,description="Include Rank Control?", 
                                         style=style, layout=layout),
           include_perf_control=Checkbox(value=True,description="Include Performance Control?", 
                                         style=style, layout=pad),
            
           discrim=FloatText(value=10000, description='Additional Pay to Men', style=style, layout=layout),
           rs=IntText(value=8675309, description='Random Seed Value', style=style, layout=layout))

PEwidget2 = interact_manual(base_pay = IntText(value=90000, description='Base Pay', 
                                              style=style, layout=layout),
           epsilon = FloatText(value=20000, step=0.01,
                                      description='Noise in Pay (mean of random error; should be positive or zero)', 
                                      style=style, layout=pad),
                 
           n_men_job_1=IntSlider(value=100, min=0, max=10000, 
                                 description='N Men in Job 1', 
                                 style=style, layout=pad), 
           n_women_job_1=IntSlider(value=100, min=0, max=10000, 
                                   description='N Women in Job 1', style=style, 
                                   layout=layout),

           n_men_job_2 = IntSlider(value=50, min=0, max=10000, 
                                   description='N Men in Job 2', style=style, 
                                   layout=layout), 
           n_women_job_2 = IntSlider(value=50, min=0, max=10000, 
                                     description='N Women in Job 2', 
                                     style=style, layout=layout),
           extra_pay_job_2=FloatText(value=30000, 
                                     description='Pay Bonus for Job 2 (relative to Job 1)', 
                                     style=style, layout=pad),

           n_men_job_3 = IntSlider(value=20, min=0, max=10000, 
                                   description='N Men in Job 3', style=style, 
                                   layout=layout), 
           n_women_job_3 = IntSlider(value=20, min=0, max=10000, 
                                     description='N Women in Job 3', 
                                     style=style, layout=layout),
           extra_pay_job_3=FloatText(value=40000, style=style, layout=pad,
                                     description='Pay Bonus for Job 3 (relative to Job 1)'),

           p_men_A_j1=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 1 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j1=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 1 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j1=FloatText(value=20000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 1'),

           p_men_A_j2=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 2 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j2=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 2 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j2=FloatText(value=20000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 2'),

           p_men_A_j3=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 3 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j3=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 3 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j3=FloatText(value=20000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 3'),

           eval_men_A_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank A'),
           eval_men_A_sd = FloatText(value=1,style=style, layout=layout,description='Noise in Performance for Men in Rank A (standard deviation; should be positive)'),
           eval_women_A_mu=FloatText(value=0, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank A'),
           eval_women_A_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank A (standard deviation; should be positive)'),
           extra_pay_eval_A =FloatText(value=100, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for A Rank'),

           eval_men_B_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank B'),
           eval_men_B_sd=FloatText(value=1, 
                                          style=style, layout=layout,
                                          description='Noise in Performance for Men in Rank B (standard deviation; should be positive)'),
           eval_women_B_mu=FloatText(value=0, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank B'),
           eval_women_B_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank B (standard deviation; should be positive)'),
           extra_pay_eval_B =FloatText(value=100, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for B Rank'),

           include_job_controls=Checkbox(value=True,description="Include Job Controls?", 
                                         style=style, layout=layout),
           include_rank_control=Checkbox(value=True,description="Include Rank Control?", 
                                         style=style, layout=layout),
           include_perf_control=Checkbox(value=True,description="Include Performance Control?", 
                                         style=style, layout=pad),
            
           discrim=FloatText(value=2000, description='Additional Pay to Men', style=style, layout=layout),
           rs=IntText(value=8675309, description='Random Seed Value', style=style, layout=layout))

PEwidget3 = interact_manual(base_pay = IntText(value=50000, description='Base Pay', 
                                              style=style, layout=layout),
           epsilon = FloatText(value=1000, step=0.01,
                                      description='Noise in Pay (mean of random error; should be positive or zero)', 
                                      style=style, layout=pad),
                 
           n_men_job_1=IntSlider(value=75, min=0, max=10000, 
                                 description='N Men in Job 1', 
                                 style=style, layout=pad), 
           n_women_job_1=IntSlider(value=75, min=0, max=10000, 
                                   description='N Women in Job 1', style=style, 
                                   layout=layout),

           n_men_job_2 = IntSlider(value=75, min=0, max=10000, 
                                   description='N Men in Job 2', style=style, 
                                   layout=layout), 
           n_women_job_2 = IntSlider(value=75, min=0, max=10000, 
                                     description='N Women in Job 2', 
                                     style=style, layout=layout),
           extra_pay_job_2=FloatText(value=0, 
                                     description='Pay Bonus for Job 2 (relative to Job 1)', 
                                     style=style, layout=pad),

           n_men_job_3 = IntSlider(value=60, min=0, max=10000, 
                                   description='N Men in Job 3', style=style, 
                                   layout=layout), 
           n_women_job_3 = IntSlider(value=20, min=0, max=10000, 
                                     description='N Women in Job 3', 
                                     style=style, layout=layout),
           extra_pay_job_3=FloatText(value=10000, style=style, layout=pad,
                                     description='Pay Bonus for Job 3 (relative to Job 1)'),

           p_men_A_j1=FloatSlider(value=0.15, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 1 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j1=FloatSlider(value=0.05, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 1 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j1=FloatText(value=10000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 1'),

           p_men_A_j2=FloatSlider(value=0.15, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 2 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j2=FloatSlider(value=0.05, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 2 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j2=FloatText(value=10000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 2'),

           p_men_A_j3=FloatSlider(value=0.25, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 3 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j3=FloatSlider(value=0.25, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 3 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j3=FloatText(value=10000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 3'),

           eval_men_A_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank A'),
           eval_men_A_sd = FloatText(value=1,style=style, layout=layout,description='Noise in Performance for Men in Rank A (standard deviation; should be positive)'),
           eval_women_A_mu=FloatText(value=0, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank A'),
           eval_women_A_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank A (standard deviation; should be positive)'),
           extra_pay_eval_A =FloatText(value=10000, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for A Rank'),

           eval_men_B_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank B'),
           eval_men_B_sd=FloatText(value=1, 
                                          style=style, layout=layout,
                                          description='Noise in Performance for Men in Rank B (standard deviation; should be positive)'),
           eval_women_B_mu=FloatText(value=0, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank B'),
           eval_women_B_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank B (standard deviation; should be positive)'),
           extra_pay_eval_B =FloatText(value=10000, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for B Rank'),

           include_job_controls=Checkbox(value=True,description="Include Job Controls?", 
                                         style=style, layout=layout),
           include_rank_control=Checkbox(value=True,description="Include Rank Control?", 
                                         style=style, layout=layout),
           include_perf_control=Checkbox(value=True,description="Include Performance Control?", 
                                         style=style, layout=pad),
            
           discrim=FloatText(value=0, description='Additional Pay to Men', style=style, layout=layout),
           rs=IntText(value=8675309, description='Random Seed Value', style=style, layout=layout))

PEwidget4 = interact_manual(base_pay = IntText(value=60000, description='Base Pay', 
                                              style=style, layout=layout),
           epsilon = FloatText(value=1000, step=0.01,
                                      description='Noise in Pay (mean of random error; should be positive or zero)', 
                                      style=style, layout=pad),
                 
           n_men_job_1=IntSlider(value=300, min=0, max=10000, 
                                 description='N Men in Job 1', 
                                 style=style, layout=pad), 
           n_women_job_1=IntSlider(value=200, min=0, max=10000, 
                                   description='N Women in Job 1', style=style, 
                                   layout=layout),

           n_men_job_2 = IntSlider(value=300, min=0, max=10000, 
                                   description='N Men in Job 2', style=style, 
                                   layout=layout), 
           n_women_job_2 = IntSlider(value=250, min=0, max=10000, 
                                     description='N Women in Job 2', 
                                     style=style, layout=layout),
           extra_pay_job_2=FloatText(value=2500, 
                                     description='Pay Bonus for Job 2 (relative to Job 1)', 
                                     style=style, layout=pad),

           n_men_job_3 = IntSlider(value=30, min=0, max=10000, 
                                   description='N Men in Job 3', style=style, 
                                   layout=layout), 
           n_women_job_3 = IntSlider(value=45, min=0, max=10000, 
                                     description='N Women in Job 3', 
                                     style=style, layout=layout),
           extra_pay_job_3=FloatText(value=25000, style=style, layout=pad,
                                     description='Pay Bonus for Job 3 (relative to Job 1)'),

           p_men_A_j1=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 1 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j1=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 1 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j1=FloatText(value=10000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 1'),

           p_men_A_j2=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 2 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j2=FloatSlider(value=0.3, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 2 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j2=FloatText(value=10000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 2'),

           p_men_A_j3=FloatSlider(value=0.1, min=0, max=1, step=0.01, 
                                  description='Proportion of Men in Job 3 Who Are Rank A', 
                                  style=style, layout=layout),
           p_women_A_j3=FloatSlider(value=0.1, min=0, max=1, step=0.01, 
                                    description='Proportion of Women in Job 3 Who Are Rank A', 
                                    style=style, layout=layout),
           extra_pay_A_j3=FloatText(value=20000, style=style, layout=pad,
                                    description='Pay Bonus for A Rank in Job 3'),

           eval_men_A_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank A'),
           eval_men_A_sd = FloatText(value=1,style=style, layout=layout,description='Noise in Performance for Men in Rank A (standard deviation; should be positive)'),
           eval_women_A_mu=FloatText(value=1, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank A'),
           eval_women_A_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank A (standard deviation; should be positive)'),
           extra_pay_eval_A =FloatText(value=5000, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for A Rank'),

           eval_men_B_mu=FloatText(value=0, style=style, layout=layout,
                                   description='Average Performance for Men who are Rank B'),
           eval_men_B_sd=FloatText(value=1, 
                                          style=style, layout=layout,
                                          description='Noise in Performance for Men in Rank B (standard deviation; should be positive)'),
           eval_women_B_mu=FloatText(value=1, style=style, layout=layout,
                                     description='Average Performance for Women who are Rank B'),
           eval_women_B_sd=FloatText(value=1, style=style, layout=layout,
                                            description='Noise in Performance for Women in Rank B (standard deviation; should be positive)'),
           extra_pay_eval_B =FloatText(value=10000, style=style, layout=pad,
                                       description='Pay Bonus for Each 1-Point Increase in Performance for B Rank'),

           include_job_controls=Checkbox(value=True,description="Include Job Controls?", 
                                         style=style, layout=layout),
           include_rank_control=Checkbox(value=True,description="Include Rank Control?", 
                                         style=style, layout=layout),
           include_perf_control=Checkbox(value=False,description="Include Performance Control?", 
                                         style=style, layout=pad),
            
           discrim=FloatText(value=1000, description='Additional Pay to Men', style=style, layout=layout),
           rs=IntText(value=8675309, description='Random Seed Value', style=style, layout=layout))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Scenario 1

In [ ]:
@PEwidget1
def sim(base_pay = 30000,
        epsilon = 4000,
        n_women_job_1=80, n_men_job_1=120,
        n_women_job_2=90, n_men_job_2=110, extra_pay_job_2=10000,
        n_women_job_3=130, n_men_job_3=70, extra_pay_job_3=20000,
        p_women_A_j1=0.5,p_men_A_j1=0.3,extra_pay_A_j1=15000,
        p_women_A_j2=0.5,p_men_A_j2=0.3,extra_pay_A_j2=15000,
        p_women_A_j3=0.5,p_men_A_j3=0.3,extra_pay_A_j3=15000,
        eval_women_A_mu=1, eval_women_A_sd=1,
        eval_men_A_mu=0,  eval_men_A_sd=1, 
        extra_pay_eval_A = 5000,
        eval_women_B_mu=1, eval_women_B_sd=1,
        eval_men_B_mu=0,  eval_men_B_sd=1, 
        extra_pay_eval_B = 5000,
        include_job_controls=True,
        include_rank_control=True,
        include_perf_control=True,
        discrim=10000,
        rs=8675309):
  
  np.random.seed(int(rs))

  # Make variables that determine pay
  Male = []
  Job_2 = []
  Job_3 = []
  Rank_A = []
  Performance = []

  # Simulate the Data
  for j in [1,2,3]:
    for r in ['A', 'B']:

      #Get relevant counts and proportions from parameters
      w = eval('n_women_job_{}'.format(j))
      w_p = eval('p_women_A_j{}'.format(j))
      m = eval('n_men_job_{}'.format(j))
      m_p = eval('p_men_A_j{}'.format(j))     

      if r == "A":
        n_women = round(w * w_p)
        n_men = round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [1] * n_total

        Performance += np.random.normal(loc=int(eval_women_A_mu), 
                                        scale=int(eval_women_A_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_A_mu), 
                                        scale=int(eval_men_A_sd), 
                                        size=n_men).tolist()

      else:
        n_women = w - round(w * w_p)
        n_men = m - round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [0] * n_total

        Performance += np.random.normal(loc=int(eval_women_B_mu), 
                                        scale=int(eval_women_B_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_B_mu), 
                                        scale=int(eval_men_B_sd), 
                                        size=n_men).tolist()

  # Combine simulated data into dataframe
  df = pd.DataFrame(data={"Male": Male, "Job_2": Job_2, 
                          "Performance": Performance, "Job_3": Job_3, 
                          "Rank_A": Rank_A})

  df['Pay'] = int(base_pay)
  df['Pay'] += np.random.normal(scale=float(epsilon), size=len(df))

  df['Pay'] +=  df.Male * float(discrim)

  df['Pay'] +=  df.Job_2 * float(extra_pay_job_2)
  df['Pay'] +=  df.Job_3 * float(extra_pay_job_3)

  df['Pay'] +=  (1 - df.Job_2 - df.Job_3) * df.Rank_A * float(extra_pay_A_j1)
  df['Pay'] +=  df.Job_2 * df.Rank_A * float(extra_pay_A_j2)
  df['Pay'] +=  df.Job_3 * df.Rank_A * float(extra_pay_A_j3)

  df['Pay'] +=  df.Rank_A * df.Performance * float(extra_pay_eval_A)
  df['Pay'] +=  (1 - df.Rank_A) * df.Performance * float(extra_pay_eval_B)

  # Unadjusted Pay Gap
  print("\nUNADJUSTED PAY GAP")
  print("------------------")
  sb.set_style('whitegrid')
  sb.histplot(df[df['Male'] == 1].Pay,label='Men',color=my_blue, alpha=0.4,kde=True)
  sb.histplot(df[df['Male'] == 0].Pay,label='Women',color=my_orange, alpha=0.4,kde=True)
  plt.legend()
  plt.show()

  men_pay = df[df.Male == 1].Pay
  women_pay = df[df.Male == 0].Pay
  print("Median pay for women: {}".format(round(women_pay.median(),2)))
  print("Median pay for men: {}".format(round(men_pay.median(),2)))
  unadjusted = round((1 - (women_pay.median()/men_pay.median())) * 100, 2)
  print("Unadjusted Pay Gap: {}%".format(unadjusted))
  t, p = stats.ttest_ind(men_pay, women_pay)
  print(r"P-value for the difference in means: {}".format(p))

  # Descriptive Graphs
  print("\nDESCRIPTIVE GRAPHS")
  print("------------------") 

  df['Job'] = 1 + df.Job_2 + (2*df.Job_3)

  sb.set(style="whitegrid")
  f, ax = plt.subplots(figsize=(10, 5))
  sb.barplot(x='Job', y='Male', data=df, label="Women", color=my_orange,
            estimator=lambda x: 100, ci=None)
  sb.barplot(x='Job', y='Male', data=df, label="Men", color=my_blue,
            estimator=lambda x: (sum(x==1)*100.0)/len(x), ci=None)
  plt.legend(loc="lower left")
  plt.ylabel("% Composition")
  sb.despine(top=True, left=True)
  plt.axhline((sum(df.Male)*100)/len(df), color='black', ls= '--')
  plt.show() 

  g = sb.catplot(x="Male", y="Rank_A", col='Job', data=df, 
                palette={0: my_orange,1: my_blue},
                kind="bar", ci=False, estimator=lambda x: np.mean(x)*100)
  (g.set_axis_labels("", "% A Ranked")
  .set_xticklabels(["Women", "Men"])
  .set_titles("{col_var} {col_name}")
  .set(ylim=(0, 100)))
  plt.show()

  if len(df) <= 6000:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.8}, 
                palette={0: my_orange, 1: my_blue})
  else:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.1}, 
                palette={0: my_orange, 1: my_blue})
  plt.show()


  # OLS REGRESSION
  print("\nOLS REGRESSION")
  print("--------------") 

  X = ['Male']
  if include_job_controls:
    X += ['Job_2', 'Job_3']
  if include_rank_control:
    X.append("Rank_A")
  if include_perf_control:
    X.append("Performance")

  m = sm.OLS(endog=df.Pay, exog=sm.add_constant(df[X]))
  res = m.fit()
  print(res.summary())
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

interactive(children=(IntText(value=30000, description='Base Pay', layout=Layout(width='1000px'), style=Descri…

# Scenario 2

In [ ]:
@PEwidget2
def sim(base_pay = 30000,
        epsilon = 4000,
        n_women_job_1=80, n_men_job_1=120,
        n_women_job_2=90, n_men_job_2=110, extra_pay_job_2=10000,
        n_women_job_3=130, n_men_job_3=70, extra_pay_job_3=20000,
        p_women_A_j1=0.5,p_men_A_j1=0.3,extra_pay_A_j1=15000,
        p_women_A_j2=0.5,p_men_A_j2=0.3,extra_pay_A_j2=15000,
        p_women_A_j3=0.5,p_men_A_j3=0.3,extra_pay_A_j3=15000,
        eval_women_A_mu=1, eval_women_A_sd=1,
        eval_men_A_mu=0,  eval_men_A_sd=1, 
        extra_pay_eval_A = 5000,
        eval_women_B_mu=1, eval_women_B_sd=1,
        eval_men_B_mu=0,  eval_men_B_sd=1, 
        extra_pay_eval_B = 5000,
        include_job_controls=True,
        include_rank_control=True,
        include_perf_control=True,
        discrim=10000,
        rs=8675309):
  
  np.random.seed(int(rs))

  # Make variables that determine pay
  Male = []
  Job_2 = []
  Job_3 = []
  Rank_A = []
  Performance = []

  # Simulate the Data
  for j in [1,2,3]:
    for r in ['A', 'B']:

      #Get relevant counts and proportions from parameters
      w = eval('n_women_job_{}'.format(j))
      w_p = eval('p_women_A_j{}'.format(j))
      m = eval('n_men_job_{}'.format(j))
      m_p = eval('p_men_A_j{}'.format(j))     

      if r == "A":
        n_women = round(w * w_p)
        n_men = round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [1] * n_total

        Performance += np.random.normal(loc=int(eval_women_A_mu), 
                                        scale=int(eval_women_A_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_A_mu), 
                                        scale=int(eval_men_A_sd), 
                                        size=n_men).tolist()

      else:
        n_women = w - round(w * w_p)
        n_men = m - round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [0] * n_total

        Performance += np.random.normal(loc=int(eval_women_B_mu), 
                                        scale=int(eval_women_B_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_B_mu), 
                                        scale=int(eval_men_B_sd), 
                                        size=n_men).tolist()

  # Combine simulated data into dataframe
  df = pd.DataFrame(data={"Male": Male, "Job_2": Job_2, 
                          "Performance": Performance, "Job_3": Job_3, 
                          "Rank_A": Rank_A})

  df['Pay'] = int(base_pay)
  df['Pay'] += np.random.normal(scale=float(epsilon), size=len(df))

  df['Pay'] +=  df.Male * float(discrim)

  df['Pay'] +=  df.Job_2 * float(extra_pay_job_2)
  df['Pay'] +=  df.Job_3 * float(extra_pay_job_3)

  df['Pay'] +=  (1 - df.Job_2 - df.Job_3) * df.Rank_A * float(extra_pay_A_j1)
  df['Pay'] +=  df.Job_2 * df.Rank_A * float(extra_pay_A_j2)
  df['Pay'] +=  df.Job_3 * df.Rank_A * float(extra_pay_A_j3)

  df['Pay'] +=  df.Rank_A * df.Performance * float(extra_pay_eval_A)
  df['Pay'] +=  (1 - df.Rank_A) * df.Performance * float(extra_pay_eval_B)

  # Unadjusted Pay Gap
  print("\nUNADJUSTED PAY GAP")
  print("------------------")
  sb.set_style('whitegrid')
  sb.histplot(df[df['Male'] == 1].Pay,label='Men',color=my_blue, alpha=0.4,kde=True)
  sb.histplot(df[df['Male'] == 0].Pay,label='Women',color=my_orange, alpha=0.4,kde=True)
  plt.legend()
  plt.show()

  men_pay = df[df.Male == 1].Pay
  women_pay = df[df.Male == 0].Pay
  print("Median pay for women: {}".format(round(women_pay.median(),2)))
  print("Median pay for men: {}".format(round(men_pay.median(),2)))
  unadjusted = round((1 - (women_pay.median()/men_pay.median())) * 100, 2)
  print("Unadjusted Pay Gap: {}%".format(unadjusted))
  t, p = stats.ttest_ind(men_pay, women_pay)
  print(r"P-value for the difference in means: {}".format(p))

  # Descriptive Graphs
  print("\nDESCRIPTIVE GRAPHS")
  print("------------------") 

  df['Job'] = 1 + df.Job_2 + (2*df.Job_3)

  sb.set(style="whitegrid")
  f, ax = plt.subplots(figsize=(10, 5))
  sb.barplot(x='Job', y='Male', data=df, label="Women", color=my_orange,
            estimator=lambda x: 100, ci=None)
  sb.barplot(x='Job', y='Male', data=df, label="Men", color=my_blue,
            estimator=lambda x: (sum(x==1)*100.0)/len(x), ci=None)
  plt.legend(loc="lower left")
  plt.ylabel("% Composition")
  sb.despine(top=True, left=True)
  plt.axhline((sum(df.Male)*100)/len(df), color='black', ls= '--')
  plt.show() 

  g = sb.catplot(x="Male", y="Rank_A", col='Job', data=df, 
                palette={0: my_orange,1: my_blue},
                kind="bar", ci=False, estimator=lambda x: np.mean(x)*100)
  (g.set_axis_labels("", "% A Ranked")
  .set_xticklabels(["Women", "Men"])
  .set_titles("{col_var} {col_name}")
  .set(ylim=(0, 100)))
  plt.show()

  if len(df) <= 6000:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.8}, 
                palette={0: my_orange, 1: my_blue})
  else:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.1}, 
                palette={0: my_orange, 1: my_blue})
  plt.show()


  # OLS REGRESSION
  print("\nOLS REGRESSION")
  print("--------------") 

  X = ['Male']
  if include_job_controls:
    X += ['Job_2', 'Job_3']
  if include_rank_control:
    X.append("Rank_A")
  if include_perf_control:
    X.append("Performance")

  m = sm.OLS(endog=df.Pay, exog=sm.add_constant(df[X]))
  res = m.fit()
  print(res.summary())
  

# Scenario 3

In [ ]:
@PEwidget3
def sim(base_pay = 50000,
        epsilon = 1000,
        n_women_job_1=500, n_men_job_1=500,
        n_women_job_2=500, n_men_job_2=500, extra_pay_job_2=0,
        n_women_job_3=200, n_men_job_3=800, extra_pay_job_3=10000,
        p_women_A_j1=0.1,p_men_A_j1=0.15,extra_pay_A_j1=10000,
        p_women_A_j2=0.1,p_men_A_j2=0.15,extra_pay_A_j2=10000,
        p_women_A_j3=0.25,p_men_A_j3=0.25,extra_pay_A_j3=10000,
        eval_women_A_mu=0, eval_women_A_sd=1,
        eval_men_A_mu=0,  eval_men_A_sd=1, 
        extra_pay_eval_A = 10000,
        eval_women_B_mu=0, eval_women_B_sd=1,
        eval_men_B_mu=0,  eval_men_B_sd=1, 
        extra_pay_eval_B = 10000,
        include_job_controls=True,
        include_rank_control=True,
        include_perf_control=True,
        discrim=0,
        rs=8675309):
  
  np.random.seed(int(rs))

  # Make variables that determine pay
  Male = []
  Job_2 = []
  Job_3 = []
  Rank_A = []
  Performance = []

  # Simulate the Data
  for j in [1,2,3]:
    for r in ['A', 'B']:

      #Get relevant counts and proportions from parameters
      w = eval('n_women_job_{}'.format(j))
      w_p = eval('p_women_A_j{}'.format(j))
      m = eval('n_men_job_{}'.format(j))
      m_p = eval('p_men_A_j{}'.format(j))     

      if r == "A":
        n_women = round(w * w_p)
        n_men = round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [1] * n_total

        Performance += np.random.normal(loc=int(eval_women_A_mu), 
                                        scale=int(eval_women_A_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_A_mu), 
                                        scale=int(eval_men_A_sd), 
                                        size=n_men).tolist()

      else:
        n_women = w - round(w * w_p)
        n_men = m - round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [0] * n_total

        Performance += np.random.normal(loc=int(eval_women_B_mu), 
                                        scale=int(eval_women_B_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_B_mu), 
                                        scale=int(eval_men_B_sd), 
                                        size=n_men).tolist()

  # Combine simulated data into dataframe
  df = pd.DataFrame(data={"Male": Male, "Job_2": Job_2, 
                          "Performance": Performance, "Job_3": Job_3, 
                          "Rank_A": Rank_A})

  df['Pay'] = int(base_pay)
  df['Pay'] += np.random.normal(scale=float(epsilon), size=len(df))

  df['Pay'] +=  df.Male * float(discrim)

  df['Pay'] +=  df.Job_2 * float(extra_pay_job_2)
  df['Pay'] +=  df.Job_3 * float(extra_pay_job_3)

  df['Pay'] +=  (1 - df.Job_2 - df.Job_3) * df.Rank_A * float(extra_pay_A_j1)
  df['Pay'] +=  df.Job_2 * df.Rank_A * float(extra_pay_A_j2)
  df['Pay'] +=  df.Job_3 * df.Rank_A * float(extra_pay_A_j3)

  df['Pay'] +=  df.Rank_A * df.Performance * float(extra_pay_eval_A)
  df['Pay'] +=  (1 - df.Rank_A) * df.Performance * float(extra_pay_eval_B)

  # Unadjusted Pay Gap
  print("\nUNADJUSTED PAY GAP")
  print("------------------")
  sb.set_style('whitegrid')
  sb.histplot(df[df['Male'] == 1].Pay,label='Men',color=my_blue, alpha=0.4,kde=True)
  sb.histplot(df[df['Male'] == 0].Pay,label='Women',color=my_orange, alpha=0.4,kde=True)
  plt.legend()
  plt.show()

  men_pay = df[df.Male == 1].Pay
  women_pay = df[df.Male == 0].Pay
  print("Median pay for women: {}".format(round(women_pay.median(),2)))
  print("Median pay for men: {}".format(round(men_pay.median(),2)))
  unadjusted = round((1 - (women_pay.median()/men_pay.median())) * 100, 2)
  print("Unadjusted Pay Gap: {}%".format(unadjusted))
  t, p = stats.ttest_ind(men_pay, women_pay)
  print(r"P-value for the difference in means: {}".format(p))

  # Descriptive Graphs
  print("\nDESCRIPTIVE GRAPHS")
  print("------------------") 

  df['Job'] = 1 + df.Job_2 + (2*df.Job_3)

  sb.set(style="whitegrid")
  f, ax = plt.subplots(figsize=(10, 5))
  sb.barplot(x='Job', y='Male', data=df, label="Women", color=my_orange,
            estimator=lambda x: 100, ci=None)
  sb.barplot(x='Job', y='Male', data=df, label="Men", color=my_blue,
            estimator=lambda x: (sum(x==1)*100.0)/len(x), ci=None)
  plt.legend(loc="lower left")
  plt.ylabel("% Composition")
  sb.despine(top=True, left=True)
  plt.axhline((sum(df.Male)*100)/len(df), color='black', ls= '--')
  plt.show() 

  g = sb.catplot(x="Male", y="Rank_A", col='Job', data=df, 
                palette={0: my_orange,1: my_blue},
                kind="bar", ci=False, estimator=lambda x: np.mean(x)*100)
  (g.set_axis_labels("", "% A Ranked")
  .set_xticklabels(["Women", "Men"])
  .set_titles("{col_var} {col_name}")
  .set(ylim=(0, 100)))
  plt.show()

  if len(df) <= 6000:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.8}, 
                palette={0: my_orange, 1: my_blue})
  else:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.1}, 
                palette={0: my_orange, 1: my_blue})
  plt.show()


  # OLS REGRESSION
  print("\nOLS REGRESSION")
  print("--------------") 

  X = ['Male']
  if include_job_controls:
    X += ['Job_2', 'Job_3']
  if include_rank_control:
    X.append("Rank_A")
  if include_perf_control:
    X.append("Performance")

  m = sm.OLS(endog=df.Pay, exog=sm.add_constant(df[X]))
  res = m.fit()
  print(res.summary())
  

# Scenario 4

In [ ]:
@PEwidget4
def sim(base_pay = 50000,
        epsilon = 1000,
        n_women_job_1=500, n_men_job_1=500,
        n_women_job_2=500, n_men_job_2=500, extra_pay_job_2=0,
        n_women_job_3=200, n_men_job_3=800, extra_pay_job_3=10000,
        p_women_A_j1=0.1,p_men_A_j1=0.15,extra_pay_A_j1=10000,
        p_women_A_j2=0.1,p_men_A_j2=0.15,extra_pay_A_j2=10000,
        p_women_A_j3=0.25,p_men_A_j3=0.25,extra_pay_A_j3=10000,
        eval_women_A_mu=0, eval_women_A_sd=1,
        eval_men_A_mu=0,  eval_men_A_sd=1, 
        extra_pay_eval_A = 10000,
        eval_women_B_mu=0, eval_women_B_sd=1,
        eval_men_B_mu=0,  eval_men_B_sd=1, 
        extra_pay_eval_B = 10000,
        include_job_controls=True,
        include_rank_control=True,
        include_perf_control=True,
        discrim=0,
        rs=8675309):
  
  np.random.seed(int(rs))

  # Make variables that determine pay
  Male = []
  Job_2 = []
  Job_3 = []
  Rank_A = []
  Performance = []

  # Simulate the Data
  for j in [1,2,3]:
    for r in ['A', 'B']:

      #Get relevant counts and proportions from parameters
      w = eval('n_women_job_{}'.format(j))
      w_p = eval('p_women_A_j{}'.format(j))
      m = eval('n_men_job_{}'.format(j))
      m_p = eval('p_men_A_j{}'.format(j))     

      if r == "A":
        n_women = round(w * w_p)
        n_men = round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [1] * n_total

        Performance += np.random.normal(loc=int(eval_women_A_mu), 
                                        scale=int(eval_women_A_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_A_mu), 
                                        scale=int(eval_men_A_sd), 
                                        size=n_men).tolist()

      else:
        n_women = w - round(w * w_p)
        n_men = m - round(m * m_p)
        n_total = n_women + n_men

        Male += [0] * n_women
        Male += [1] * n_men

        if j == 2:
          Job_2 += [1] * n_total
        else:
          Job_2 += [0] * n_total

        if j == 3:
          Job_3 += [1] * n_total
        else:
          Job_3 += [0] * n_total

        Rank_A += [0] * n_total

        Performance += np.random.normal(loc=int(eval_women_B_mu), 
                                        scale=int(eval_women_B_sd), 
                                        size=n_women).tolist()
        Performance += np.random.normal(loc=int(eval_men_B_mu), 
                                        scale=int(eval_men_B_sd), 
                                        size=n_men).tolist()

  # Combine simulated data into dataframe
  df = pd.DataFrame(data={"Male": Male, "Job_2": Job_2, 
                          "Performance": Performance, "Job_3": Job_3, 
                          "Rank_A": Rank_A})

  df['Pay'] = int(base_pay)
  df['Pay'] += np.random.normal(scale=float(epsilon), size=len(df))

  df['Pay'] +=  df.Male * float(discrim)

  df['Pay'] +=  df.Job_2 * float(extra_pay_job_2)
  df['Pay'] +=  df.Job_3 * float(extra_pay_job_3)

  df['Pay'] +=  (1 - df.Job_2 - df.Job_3) * df.Rank_A * float(extra_pay_A_j1)
  df['Pay'] +=  df.Job_2 * df.Rank_A * float(extra_pay_A_j2)
  df['Pay'] +=  df.Job_3 * df.Rank_A * float(extra_pay_A_j3)

  df['Pay'] +=  df.Rank_A * df.Performance * float(extra_pay_eval_A)
  df['Pay'] +=  (1 - df.Rank_A) * df.Performance * float(extra_pay_eval_B)

  # Unadjusted Pay Gap
  print("\nUNADJUSTED PAY GAP")
  print("------------------")
  sb.set_style('whitegrid')
  sb.histplot(df[df['Male'] == 1].Pay,label='Men',color=my_blue, alpha=0.4,kde=True)
  sb.histplot(df[df['Male'] == 0].Pay,label='Women',color=my_orange, alpha=0.4,kde=True)
  plt.legend()
  plt.show()

  men_pay = df[df.Male == 1].Pay
  women_pay = df[df.Male == 0].Pay
  print("Median pay for women: {}".format(round(women_pay.median(),2)))
  print("Median pay for men: {}".format(round(men_pay.median(),2)))
  unadjusted = round((1 - (women_pay.median()/men_pay.median())) * 100, 2)
  print("Unadjusted Pay Gap: {}%".format(unadjusted))
  t, p = stats.ttest_ind(men_pay, women_pay)
  print(r"P-value for the difference in means: {}".format(p))

  # Descriptive Graphs
  print("\nDESCRIPTIVE GRAPHS")
  print("------------------") 

  df['Job'] = 1 + df.Job_2 + (2*df.Job_3)

  sb.set(style="whitegrid")
  f, ax = plt.subplots(figsize=(10, 5))
  sb.barplot(x='Job', y='Male', data=df, label="Women", color=my_orange,
            estimator=lambda x: 100, ci=None)
  sb.barplot(x='Job', y='Male', data=df, label="Men", color=my_blue,
            estimator=lambda x: (sum(x==1)*100.0)/len(x), ci=None)
  plt.legend(loc="lower left")
  plt.ylabel("% Composition")
  sb.despine(top=True, left=True)
  plt.axhline((sum(df.Male)*100)/len(df), color='black', ls= '--')
  plt.show() 

  g = sb.catplot(x="Male", y="Rank_A", col='Job', data=df, 
                palette={0: my_orange,1: my_blue},
                kind="bar", ci=False, estimator=lambda x: np.mean(x)*100)
  (g.set_axis_labels("", "% A Ranked")
  .set_xticklabels(["Women", "Men"])
  .set_titles("{col_var} {col_name}")
  .set(ylim=(0, 100)))
  plt.show()

  if len(df) <= 6000:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.8}, 
                palette={0: my_orange, 1: my_blue})
  else:
    sb.set_style("white")
    sb.jointplot(data=df, x="Performance", y="Pay", kind="scatter", 
                hue="Male", joint_kws={"alpha":0.1}, 
                palette={0: my_orange, 1: my_blue})
  plt.show()


  # OLS REGRESSION
  print("\nOLS REGRESSION")
  print("--------------") 

  X = ['Male']
  if include_job_controls:
    X += ['Job_2', 'Job_3']
  if include_rank_control:
    X.append("Rank_A")
  if include_perf_control:
    X.append("Performance")

  m = sm.OLS(endog=df.Pay, exog=sm.add_constant(df[X]))
  res = m.fit()
  print(res.summary())
  